In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [2]:
url = "https://www.bls.gov/news.release/cpi.htm"
ua = UserAgent()
header = {
    'User-Agent': ua.chrome
}

r = requests.get(url, headers=header)

In [5]:
bs = BeautifulSoup(r.content, "html.parser")

In [101]:
import datetime

update = bs.find(class_="update")
date = update.text.split(":")[1].strip()

datetime.datetime.strptime("November 1, 2021", "%B %d, %Y")

datetime.datetime(2021, 11, 1, 0, 0)

In [8]:
table1 = bs.find(id="cpipress1")

In [89]:
type(table1)

bs4.element.Tag

In [11]:
table1.find(class_= "tableTitle").text

'Table 1. Consumer Price Index for All Urban Consumers (CPI-U): U.S. city average, by expenditure category, October 2021'

## tableのrow, colを計算、headerを作成

In [59]:
thead = table1.find("thead")
max_row = 0
max_col = 0
header = [[]]
for row, tr in enumerate(thead.find_all("tr")):
    max_row = max(row + 1, max_row)
    offset = 0
    for col, th in enumerate(tr.find_all("th")):
        rspan = 1
        cspan = 1
        if "rowspan" in th.attrs:
            rspan = int(th["rowspan"])
        if "colspan" in th.attrs:
            cspan = int(th["colspan"])
        max_col = max(col + offset + cspan, max_col)
        max_row = max(row + rspan, max_row)
        offset += cspan - 1
        while not len(header) == max_row:
            header.append([])
        for r in range(row, row + rspan):
            for _ in range(col + offset, col + offset + cspan):
                header[r].append(th.text)

## tableのbodyを読み込み

In [87]:
tbody = table1.find("tbody")
trows = tbody.find_all("tr")
data = []
for row, tr in enumerate(trows):
    data.append(["" for _ in range(max_col)])
    if "class" in tr.attrs and tr["class"][0] == "sep":
        continue
    th = tr.find("th")
    off = int(th.find("p")["class"][0].replace("sub", ""))
    data[row][0] = " " * off + th.text
    offset = 0
    for col, td in enumerate(tr.find_all("td")):
        colspan = 1
        if "colspan" in td.attrs:
            colspan = int(td["colspan"])
        for ci in range(col + offset, col + offset + colspan):
            data[row][ci + 1] = td.text
        offset += colspan - 1

In [92]:
type(thead)

bs4.element.Tag

In [88]:
data

[['All items',
  '100.000',
  '260.388',
  '274.310',
  '276.589',
  '6.2',
  '0.8',
  '0.3',
  '0.4',
  '0.9'],
 [' Food',
  '13.977',
  '269.828',
  '281.506',
  '284.205',
  '5.3',
  '1.0',
  '0.4',
  '0.9',
  '0.9'],
 ['  Food at home',
  '7.716',
  '251.937',
  '262.695',
  '265.478',
  '5.4',
  '1.1',
  '0.4',
  '1.2',
  '1.0'],
 ['   Cereals and bakery products',
  '0.979',
  '284.740',
  '292.069',
  '294.625',
  '3.5',
  '0.9',
  '0.0',
  '1.1',
  '1.0'],
 ['   Meats, poultry, fish, and eggs',
  '1.827',
  '265.796',
  '293.217',
  '297.377',
  '11.9',
  '1.4',
  '0.7',
  '2.2',
  '1.7'],
 ['   Dairy and related products',
  '0.753',
  '229.155',
  '231.973',
  '233.168',
  '1.8',
  '0.5',
  '-1.0',
  '0.7',
  '0.2'],
 ['   Fruits and vegetables',
  '1.310',
  '309.071',
  '315.428',
  '318.316',
  '3.0',
  '0.9',
  '0.2',
  '0.6',
  '0.1'],
 ['   Nonalcoholic beverages and beverage materials',
  '0.918',
  '178.400',
  '184.490',
  '186.387',
  '4.5',
  '1.0',
  '1.0',
  '1.2

In [91]:
header + data

[['Expenditure category',
  'RelativeimportanceSep.2021',
  'Unadjusted indexes',
  'Unadjusted indexes',
  'Unadjusted indexes',
  'Unadjusted percent change',
  'Unadjusted percent change',
  'Seasonally adjusted percent change',
  'Seasonally adjusted percent change',
  'Seasonally adjusted percent change'],
 ['Expenditure category',
  'RelativeimportanceSep.2021',
  'Oct.2020',
  'Sep.2021',
  'Oct.2021',
  'Oct.2020-Oct.2021',
  'Sep.2021-Oct.2021',
  'Jul.2021-Aug.2021',
  'Aug.2021-Sep.2021',
  'Sep.2021-Oct.2021'],
 ['All items',
  '100.000',
  '260.388',
  '274.310',
  '276.589',
  '6.2',
  '0.8',
  '0.3',
  '0.4',
  '0.9'],
 [' Food',
  '13.977',
  '269.828',
  '281.506',
  '284.205',
  '5.3',
  '1.0',
  '0.4',
  '0.9',
  '0.9'],
 ['  Food at home',
  '7.716',
  '251.937',
  '262.695',
  '265.478',
  '5.4',
  '1.1',
  '0.4',
  '1.2',
  '1.0'],
 ['   Cereals and bakery products',
  '0.979',
  '284.740',
  '292.069',
  '294.625',
  '3.5',
  '0.9',
  '0.0',
  '1.1',
  '1.0'],
 [